In [13]:
import gym
import numpy as np 
import matplotlib.pyplot as plt
import warnings
import highway_env
warnings.filterwarnings("ignore")

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [15]:
#Create the Model
def create_model():
    model= tf.keras.models.Sequential()

    model.add(tf.keras.layers.Flatten(input_shape=(1,5,5)))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))    
    model.add(tf.keras.layers.Dense(32, activation='relu')) 
    
    model.add(tf.keras.layers.Dense(5, activation='linear'))
    
    return model

In [16]:
env= gym.make('highway-v0')

In [17]:
env.reset()

array([[ 1.        ,  1.        ,  0.5       ,  0.41666666,  0.        ],
       [ 1.        ,  0.14947079, -0.25      , -0.01908947,  0.        ],
       [ 1.        ,  0.29590344, -0.25      , -0.05523511,  0.        ],
       [ 1.        ,  0.44346616,  0.        , -0.05003664,  0.        ],
       [ 1.        ,  0.585767  ,  0.25      , -0.04453511,  0.        ]],
      dtype=float32)

In [18]:
env.config['vehicles_density']=2
env.reset()

array([[ 1.        ,  1.        ,  0.75      ,  0.41666666,  0.        ],
       [ 1.        ,  0.07480554, -0.75      , -0.03138851,  0.        ],
       [ 1.        ,  0.14842913, -0.25      , -0.05529385,  0.        ],
       [ 1.        ,  0.22054733,  0.        , -0.04268331,  0.        ],
       [ 1.        ,  0.2824425 , -0.25      , -0.06115735,  0.        ]],
      dtype=float32)

In [19]:
env.config['vehicles_density']

2

In [20]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [21]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=75000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=5, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [10]:
# model= create_model()
# dqn = build_agent(model, 5)
# dqn.compile(tf.keras.optimizers.Adam(lr=1e-3), metrics=['mae'])
# dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)
# dqn.save_weights('dqn_weights_dense_latest2.h5f')

In [22]:
model= create_model()
dqn = build_agent(model, 5)
dqn.compile(tf.keras.optimizers.Adam(lr=1e-3), metrics=['mae'])

# dqn.load_weights('dqn_weights.h5f')          #All it wants is right lane                  25-->25-->5
# dqn.load_weights('dqn_weights2.h5f')         #Doesn't understand slowing down notion.     25-->128-->128-->64-->32-->5
dqn.load_weights('dqn_weights_dense_latest2.h5f')     #Doesn't understand slowing down notion.      25-->128-->128-->64-->32-->5
# dqn.load_weights('dqn_weights_dense2.h5f')   #Worst, doesn't do anything.                 25-->256-->256-->5
# dqn.load_weights('dqn_weights_dense007.h5f') #Very bad, doesn't do anything               25-->128-->128-->64-->32-->5


In [24]:
dqn.test(env,nb_episodes=20,visualize=True)
env.close()

Testing for 20 episodes ...
Episode 1: reward: 7.250, steps: 11
Episode 2: reward: 28.183, steps: 40
Episode 3: reward: 5.116, steps: 8
Episode 4: reward: 6.250, steps: 9
Episode 5: reward: 14.827, steps: 22
Episode 6: reward: 11.094, steps: 16
Episode 7: reward: 5.583, steps: 9
Episode 8: reward: 11.805, steps: 17
Episode 9: reward: 20.783, steps: 31


KeyboardInterrupt: 

In [25]:
env.close()